In [1]:
from glob import glob
import os

In [2]:
for file_path in glob("../cropped_data/*.png")[53:]:
    directory, file = file_path.split('\\')
    file_name, file_ext = file.split('.')
    new_file = os.path.join(directory, f"{str(int(file_name) - 1).zfill(4)}.{file_ext}")
    os.rename(file_path, new_file)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: '../cropped_data\\0053.png' -> '../cropped_data\\0052.png'

In [1]:
import numpy as np
from sklearn.datasets import load_digits

data, labels = load_digits(return_X_y=True)
(n_samples, n_features), n_digits = data.shape, np.unique(labels).size
print(labels)
print(
    f"# digits: {n_digits}; # samples: {n_samples}; # features {n_features}"
)

[0 1 2 ... 8 9 8]
# digits: 10; # samples: 1797; # features 64


In [2]:
from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(data, estimator[-1].labels_,
                                 metric="euclidean", sample_size=300,)
    ]

    # Show the results
    formatter_result = ("{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}"
                        "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}")
    print(formatter_result.format(*results))

In [3]:
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.decomposition import PCA
import gower

print(82 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')

kmeans = KMeans(init="k-means++", n_clusters=n_digits, n_init=4,
                random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=data, labels=labels)

kmeans = KMeans(init="random", n_clusters=n_digits, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=data, labels=labels)

pca = PCA(n_components=n_digits).fit(data)
kmeans = KMeans(init=pca.components_, n_clusters=n_digits, n_init=1)
bench_k_means(kmeans=kmeans, name="PCA-based", data=data, labels=labels)

t0 = time()
features_used = gower.gower_matrix(data)
spec = SpectralClustering(n_digits, affinity='precomputed').fit(features_used)

# Define the metrics which require only the true labels and estimator
# labels
fit_time = time() - t0
results = [name, fit_time, 0.0]

clustering_metrics = [
    metrics.homogeneity_score,
    metrics.completeness_score,
    metrics.v_measure_score,
    metrics.adjusted_rand_score,
    metrics.adjusted_mutual_info_score,
]
results += [m(labels, spec.labels_) for m in clustering_metrics]

# The silhouette score requires the full dataset
results += [
    metrics.silhouette_score(data, spec.labels_,
                             metric="euclidean", sample_size=300,)
]

# Show the results
formatter_result = ("{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}"
                    "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}")
print(formatter_result.format(*results))

print(82 * '_')

__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	0.080s	1169656	0.702	0.733	0.717	0.578	0.714	0.175
random   	0.086s	1166041	0.734	0.741	0.738	0.657	0.735	0.170
PCA-based	0.037s	1188310	0.784	0.787	0.785	0.741	0.783	0.181


NameError: name 'name' is not defined